### load data

In [1]:
from pandas import read_pickle
path = r"C:\Users\tanch\Documents\NTU\NTU Year 4\Semester 1\CZ4041 - Machine Learning\Team Project\data\full_train2.pkl"
data = read_pickle(path)
X_cols = [col for col in data.columns if col not in ["card_id","target"]]
X, y = data[X_cols], data['target']

In [2]:
def soft_clean(X):
    # drop cols with inf
    columns_with_inf = [col for col,maximum in X.max().items() if maximum == float("inf")]
    X = X.drop(columns_with_inf,axis =1)
    categorical_feature = [
        'feature1_1', 'feature1_2', 'feature1_3', 'feature1_4', 'feature1_5', 'feature2_1', 'feature2_2', 'feature2_3', 'feature3_0', 'feature3_1',
        "feature_sum","feature_max","feature_min", "quarter","feature_1","feature_2","feature_3",'first_active_month_month',
        'first_active_month_year', 'old_purchase_date_max_month',
        'old_purchase_date_max_year', 'old_purchase_date_min_month',
        'old_purchase_date_min_year','new_purchase_date_max_month', 'new_purchase_date_max_year',
        'new_purchase_date_min_month', 'new_purchase_date_min_year'
    ] 
    categorical_feature = [col for i,col in enumerate(X.columns) if col in categorical_feature]
    numerical_feature = [col for col in X.columns if col not in categorical_feature]
    # fill NAs of numeric features with mean
    X[numerical_feature] = X[numerical_feature].fillna(X[numerical_feature].mean())
    # fill NAs of categorical features with mode
    X[categorical_feature] = X[categorical_feature].apply(lambda col: col.fillna(col.mode()[0]))
    return X
X = soft_clean(X)

### commence optuna

In [3]:
from optuna.integration import XGBoostPruningCallback
from optuna import create_study
import xgboost as xgb
import pandas as pd

In [4]:
dtrain = xgb.DMatrix(X, label=y)

In [28]:
def objective(trial):

    params = { 
        "learning_rate" : 0.006772141, # trial.suggest_float("learning_rate", 0.005,0.01), # 0.005156176026887219
        "max_depth" : 6,#  trial.suggest_int('max_depth',6, 7), # 9
        "subsample" :0.78055, # trial.suggest_float("subsample", 0.75,0.85), # 0.5
        "colsample_bytree" :0.5268, # trial.suggest_float("colsample_bytree", 0.526-0.1,0.476+0.1), # 0.4
        "colsample_bylevel" :0.9212, # trial.suggest_float("colsample_bylevel", 0.84-0.1,0.84+0.1),
        "booster" :"gbtree",
        "verbosity" :1,
        "min_child_weight" : 61,
        "lambda" : 2.4932267997492037, # trial.suggest_float("lambda", 2.4932267997492037-2,2.4932267997492037+2), # 2.4932267997492037,
        "alpha" : 1.4370888707598015, # trial.suggest_float("alpha", 1.4370888707598015-1,1.4370888707598015+1), 
        "tree_method":"gpu_hist",
        "objective":"reg:squarederror",
        "eval_metric":"rmse",
        "seed":0,
    }
    pruning_callback = XGBoostPruningCallback(trial, "rmse")

    cv_scores = xgb.cv(params, dtrain, 
                       num_boost_round = 3000, nfold=3, 
                       metrics='rmse', 
                       early_stopping_rounds=20,
                       verbose_eval = True)

    return cv_scores['test-rmse-mean'].min()

In [29]:
# params = { 
#     "learning_rate" : 0.1,
#     "max_depth" :9,
#     "subsample" :0.5,
#     "colsample_bytree" :0.4,
#     "booster" :"gbtree",
#     "verbosity" :1,
#     "min_child_weight" : 61,
#     "lambda" : 2.4932267997492037,
#     "alpha" : 1.4370888707598015,
#     "tree_method":"gpu_hist",
#     "objective":"reg:squarederror",
#     "eval_metric":"rmse",
#     "seed":0,
# }

In [30]:
study = create_study(
    direction='minimize',                     
    sampler= None,          # use TPESampler if None
    pruner= None         # use median Sampler if None
)
study.optimize(objective, n_trials=10)

[I 2022-10-21 15:48:11,780] A new study created in memory with name: no-name-4b208a21-4120-4f3f-bb50-78b32710f00c


[0]	train-rmse:3.94824+0.03004	test-rmse:3.94830+0.05972
[1]	train-rmse:3.94376+0.02998	test-rmse:3.94423+0.05970
[2]	train-rmse:3.93948+0.03003	test-rmse:3.94025+0.05957
[3]	train-rmse:3.93512+0.03006	test-rmse:3.93623+0.05950
[4]	train-rmse:3.93100+0.03006	test-rmse:3.93253+0.05947
[5]	train-rmse:3.92684+0.03005	test-rmse:3.92871+0.05944
[6]	train-rmse:3.92274+0.03003	test-rmse:3.92500+0.05938
[7]	train-rmse:3.91883+0.02997	test-rmse:3.92145+0.05935
[8]	train-rmse:3.91492+0.02987	test-rmse:3.91793+0.05936
[9]	train-rmse:3.91088+0.02986	test-rmse:3.91425+0.05933
[10]	train-rmse:3.90696+0.02987	test-rmse:3.91064+0.05933
[11]	train-rmse:3.90307+0.02994	test-rmse:3.90711+0.05929
[12]	train-rmse:3.89915+0.02995	test-rmse:3.90361+0.05920
[13]	train-rmse:3.89542+0.02995	test-rmse:3.90021+0.05909
[14]	train-rmse:3.89167+0.02993	test-rmse:3.89692+0.05902
[15]	train-rmse:3.88794+0.02990	test-rmse:3.89365+0.05908
[16]	train-rmse:3.88430+0.02986	test-rmse:3.89044+0.05916
[17]	train-rmse:3.88071+

[141]	train-rmse:3.66033+0.02826	test-rmse:3.71087+0.05751
[142]	train-rmse:3.65948+0.02829	test-rmse:3.71041+0.05753
[143]	train-rmse:3.65855+0.02828	test-rmse:3.70987+0.05753
[144]	train-rmse:3.65764+0.02825	test-rmse:3.70935+0.05751
[145]	train-rmse:3.65671+0.02821	test-rmse:3.70885+0.05748
[146]	train-rmse:3.65579+0.02824	test-rmse:3.70840+0.05746
[147]	train-rmse:3.65487+0.02825	test-rmse:3.70790+0.05748
[148]	train-rmse:3.65399+0.02822	test-rmse:3.70739+0.05748
[149]	train-rmse:3.65324+0.02826	test-rmse:3.70695+0.05740
[150]	train-rmse:3.65240+0.02831	test-rmse:3.70648+0.05735
[151]	train-rmse:3.65159+0.02831	test-rmse:3.70604+0.05733
[152]	train-rmse:3.65085+0.02829	test-rmse:3.70558+0.05732
[153]	train-rmse:3.65005+0.02826	test-rmse:3.70514+0.05731
[154]	train-rmse:3.64933+0.02818	test-rmse:3.70470+0.05723
[155]	train-rmse:3.64852+0.02815	test-rmse:3.70432+0.05724
[156]	train-rmse:3.64770+0.02814	test-rmse:3.70385+0.05724
[157]	train-rmse:3.64689+0.02813	test-rmse:3.70342+0.057

[280]	train-rmse:3.58222+0.02723	test-rmse:3.67708+0.05601
[281]	train-rmse:3.58184+0.02722	test-rmse:3.67696+0.05599
[282]	train-rmse:3.58152+0.02718	test-rmse:3.67686+0.05601
[283]	train-rmse:3.58121+0.02723	test-rmse:3.67677+0.05598
[284]	train-rmse:3.58082+0.02732	test-rmse:3.67665+0.05597
[285]	train-rmse:3.58048+0.02736	test-rmse:3.67657+0.05594
[286]	train-rmse:3.58013+0.02737	test-rmse:3.67649+0.05591
[287]	train-rmse:3.57970+0.02739	test-rmse:3.67641+0.05589
[288]	train-rmse:3.57938+0.02744	test-rmse:3.67628+0.05590
[289]	train-rmse:3.57902+0.02740	test-rmse:3.67622+0.05587
[290]	train-rmse:3.57862+0.02739	test-rmse:3.67611+0.05589
[291]	train-rmse:3.57829+0.02733	test-rmse:3.67600+0.05590
[292]	train-rmse:3.57786+0.02740	test-rmse:3.67588+0.05590
[293]	train-rmse:3.57747+0.02742	test-rmse:3.67578+0.05593
[294]	train-rmse:3.57709+0.02750	test-rmse:3.67569+0.05591
[295]	train-rmse:3.57664+0.02753	test-rmse:3.67556+0.05592
[296]	train-rmse:3.57624+0.02751	test-rmse:3.67545+0.055

[419]	train-rmse:3.54091+0.02644	test-rmse:3.66806+0.05556
[420]	train-rmse:3.54068+0.02643	test-rmse:3.66804+0.05556
[421]	train-rmse:3.54049+0.02648	test-rmse:3.66800+0.05555
[422]	train-rmse:3.54025+0.02651	test-rmse:3.66796+0.05553
[423]	train-rmse:3.54003+0.02650	test-rmse:3.66792+0.05553
[424]	train-rmse:3.53983+0.02650	test-rmse:3.66787+0.05553
[425]	train-rmse:3.53961+0.02649	test-rmse:3.66783+0.05553
[426]	train-rmse:3.53936+0.02644	test-rmse:3.66779+0.05556
[427]	train-rmse:3.53915+0.02649	test-rmse:3.66775+0.05554
[428]	train-rmse:3.53893+0.02647	test-rmse:3.66771+0.05553
[429]	train-rmse:3.53871+0.02646	test-rmse:3.66767+0.05555
[430]	train-rmse:3.53846+0.02645	test-rmse:3.66764+0.05552
[431]	train-rmse:3.53825+0.02639	test-rmse:3.66761+0.05554
[432]	train-rmse:3.53806+0.02639	test-rmse:3.66758+0.05554
[433]	train-rmse:3.53780+0.02639	test-rmse:3.66752+0.05553
[434]	train-rmse:3.53751+0.02639	test-rmse:3.66747+0.05551
[435]	train-rmse:3.53726+0.02643	test-rmse:3.66743+0.055

...

[558]	train-rmse:3.50793+0.02527	test-rmse:3.66452+0.05566
[559]	train-rmse:3.50772+0.02530	test-rmse:3.66451+0.05561
[560]	train-rmse:3.50756+0.02531	test-rmse:3.66450+0.05562
[561]	train-rmse:3.50740+0.02533	test-rmse:3.66449+0.05562
[562]	train-rmse:3.50716+0.02534	test-rmse:3.66448+0.05562
[563]	train-rmse:3.50699+0.02533	test-rmse:3.66446+0.05560
[564]	train-rmse:3.50681+0.02537	test-rmse:3.66445+0.05559
[565]	train-rmse:3.50661+0.02543	test-rmse:3.66445+0.05558
[566]	train-rmse:3.50642+0.02548	test-rmse:3.66441+0.05554
[567]	train-rmse:3.50626+0.02547	test-rmse:3.66438+0.05555
[568]	train-rmse:3.50607+0.02543	test-rmse:3.66437+0.05553
[569]	train-rmse:3.50588+0.02548	test-rmse:3.66435+0.05557
[570]	train-rmse:3.50563+0.02556	test-rmse:3.66435+0.05556
[571]	train-rmse:3.50545+0.02556	test-rmse:3.66432+0.05557
[572]	train-rmse:3.50517+0.02552	test-rmse:3.66430+0.05553
[573]	train-rmse:3.50497+0.02548	test-rmse:3.66425+0.05552
[574]	train-rmse:3.50472+0.02545	test-rmse:3.66424+0.055

[697]	train-rmse:3.48093+0.02495	test-rmse:3.66279+0.05521
[698]	train-rmse:3.48073+0.02494	test-rmse:3.66279+0.05522
[699]	train-rmse:3.48055+0.02495	test-rmse:3.66278+0.05522
[700]	train-rmse:3.48033+0.02497	test-rmse:3.66276+0.05522
[701]	train-rmse:3.48013+0.02494	test-rmse:3.66274+0.05522
[702]	train-rmse:3.47993+0.02492	test-rmse:3.66273+0.05524
[703]	train-rmse:3.47978+0.02488	test-rmse:3.66274+0.05521
[704]	train-rmse:3.47957+0.02493	test-rmse:3.66273+0.05519
[705]	train-rmse:3.47934+0.02498	test-rmse:3.66270+0.05517
[706]	train-rmse:3.47920+0.02501	test-rmse:3.66269+0.05516
[707]	train-rmse:3.47899+0.02503	test-rmse:3.66271+0.05515
[708]	train-rmse:3.47883+0.02503	test-rmse:3.66269+0.05517
[709]	train-rmse:3.47861+0.02500	test-rmse:3.66267+0.05519
[710]	train-rmse:3.47841+0.02501	test-rmse:3.66268+0.05519
[711]	train-rmse:3.47822+0.02495	test-rmse:3.66266+0.05521
[712]	train-rmse:3.47810+0.02493	test-rmse:3.66264+0.05521
[713]	train-rmse:3.47793+0.02496	test-rmse:3.66264+0.055

[836]	train-rmse:3.45670+0.02476	test-rmse:3.66217+0.05504
[837]	train-rmse:3.45657+0.02476	test-rmse:3.66215+0.05504
[838]	train-rmse:3.45641+0.02473	test-rmse:3.66215+0.05505
[839]	train-rmse:3.45625+0.02472	test-rmse:3.66214+0.05505
[840]	train-rmse:3.45605+0.02481	test-rmse:3.66215+0.05505
[841]	train-rmse:3.45587+0.02485	test-rmse:3.66214+0.05506
[842]	train-rmse:3.45568+0.02490	test-rmse:3.66214+0.05506
[843]	train-rmse:3.45550+0.02491	test-rmse:3.66213+0.05505
[844]	train-rmse:3.45530+0.02495	test-rmse:3.66212+0.05505
[845]	train-rmse:3.45517+0.02494	test-rmse:3.66212+0.05505
[846]	train-rmse:3.45501+0.02498	test-rmse:3.66213+0.05505
[847]	train-rmse:3.45480+0.02495	test-rmse:3.66212+0.05505
[848]	train-rmse:3.45466+0.02495	test-rmse:3.66211+0.05505
[849]	train-rmse:3.45444+0.02499	test-rmse:3.66209+0.05506
[850]	train-rmse:3.45432+0.02499	test-rmse:3.66209+0.05507
[851]	train-rmse:3.45416+0.02499	test-rmse:3.66206+0.05509
[852]	train-rmse:3.45400+0.02496	test-rmse:3.66204+0.055

[975]	train-rmse:3.43348+0.02409	test-rmse:3.66141+0.05506
[976]	train-rmse:3.43331+0.02411	test-rmse:3.66139+0.05505
[977]	train-rmse:3.43311+0.02407	test-rmse:3.66139+0.05505
[978]	train-rmse:3.43289+0.02405	test-rmse:3.66138+0.05505
[979]	train-rmse:3.43273+0.02401	test-rmse:3.66138+0.05505
[980]	train-rmse:3.43251+0.02399	test-rmse:3.66139+0.05505
[981]	train-rmse:3.43236+0.02400	test-rmse:3.66139+0.05505
[982]	train-rmse:3.43217+0.02397	test-rmse:3.66138+0.05508
[983]	train-rmse:3.43207+0.02396	test-rmse:3.66138+0.05508
[984]	train-rmse:3.43189+0.02395	test-rmse:3.66137+0.05507
[985]	train-rmse:3.43174+0.02389	test-rmse:3.66139+0.05506
[986]	train-rmse:3.43158+0.02382	test-rmse:3.66138+0.05506
[987]	train-rmse:3.43138+0.02382	test-rmse:3.66136+0.05505
[988]	train-rmse:3.43123+0.02378	test-rmse:3.66137+0.05506
[989]	train-rmse:3.43104+0.02386	test-rmse:3.66135+0.05505
[990]	train-rmse:3.43086+0.02389	test-rmse:3.66134+0.05505
[991]	train-rmse:3.43072+0.02392	test-rmse:3.66133+0.055

[I 2022-10-21 16:11:35,104] Trial 9 finished with value: 3.6611821808009624 and parameters: {'learning_rate': 0.009450467390199148}. Best is trial 1 with value: 3.6604041121137114.


In [31]:
study.best_trial.params, study.best_trial.value

({'learning_rate': 0.006772141324857213}, 3.6604041121137114)